<a href="https://colab.research.google.com/github/james-monahan/Code-school-notebooks/blob/main/Hackathon-2/Hackathon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#imports

In [ ]:
!python -m spacy download fr_core_news_sm

In [ ]:
import numpy as np
import nltk
nltk.download('popular')
import string
from nltk.stem import SnowballStemmer
import spacy
import fr_core_news_sm
nlp = fr_core_news_sm.load()

In [2]:
import pandas as pd
from bs4 import BeautifulSoup

#data (team)

In [3]:
linkmain = 'https://raw.githubusercontent.com/parebin/bon_a_python/main/html_recette_all.csv'
df_main = pd.read_csv(linkmain, index_col=0)

In [4]:
df_main.to_csv('html_recette_all.csv')

In [ ]:
df_main['fiche_recette'] = df_main['fiche_recette'].apply(BeautifulSoup)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4213: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 4213 of the file /usr/local/lib/python3.6/dist-packages/pandas/core/series.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  mapped = lib.map_infer(values, f, convert=convert_dtype)


In [ ]:
def ajout_ingredient(soup):
  liste_ingredient =[]
  soup_ing = soup.find_all(class_ ='ingredient' )#class="recipe-ingredients__list
  for i in soup_ing :
    liste_ingredient.append(i.text.lower())
  return liste_ingredient

In [ ]:
df_main['ingredients'] = df_main['fiche_recette'].apply(ajout_ingredient)

#Clean text and create functions (my work)

In [ ]:
df_main.shape

(1046, 14)

In [ ]:
extra_stop = ['g', 'cuillères', 'cuillère', 'gramme', 'de',
 'barquette','doux','bonnee','botte','bouquet','bouteille','boîte','branche',
 'branches','brin','brins','bâton','bâtons','amer','poudre','café',"d'extrait",
 'chimique','cl','cuisse','cuisses',"entières","frais","demi",'dl','feuille','feuilles',
 "filet",'fines','tranches',"gousse","gouttes",'kg','ml',"moitié",'paquet','petit','petite',
 'petits','pincée','pincées','poignée','pot','sachet','sachets','soupe','tablette','pelées','tranche','tranches',
 "tête","verre",'verres','zeste',
]

In [ ]:
stop_words = nltk.corpus.stopwords.words('french')
stop_words.extend(extra_stop)

def remove_stop(col):
  text = []
  for val in col:
    text.append(" ".join([x for x in val.split() if x not in stop_words]))
  return text

def stemming(col):
  stem_fr = SnowballStemmer('french')
  text = []
  for val in col:
    text.append(" ".join([stem_en.stem(x) for x in val.split()]))
  return text 

def lemma(col):
  text = []
  for val in col:
    text.append(" ".join([x.lemma_ for x in nlp(val)]))
  return text 


def expand_get_dummies_todf(col, df, split = ","):
  # df[col] = df[col].apply(lambda x: x.split(split))
  df_d = pd.get_dummies(df[col].explode())
  df_d = df_d.groupby(df_d.index).sum()
  return df_d


def get_top_percent(data, col, percent, tag=None):
  if tag:
    data=data.where(data['tag_1']==tag)
  temp = expand_get_dummies_todf(col, df = data, split = ",")
  temp['group'] = 'group'
  # num = round(len(temp)*percent)#fix this, should be shape
  num = int((temp.shape[1])*percent)
  top = temp.groupby('group').sum().T['group'].nlargest(num)#.index
  # print(len(temp), temp.shape)
  return top



def find_magic(data, col, col2, search, exclude_top=.2, include_magic=.2, min_samples=50):
  filtered = data[data[col].str.contains(search)]
  temp = expand_get_dummies_todf(col2, df = filtered, split = ",")
  temp['group'] = 'group'
  num_top = int((temp.shape[1])*exclude_top)
  top = temp.groupby('group').sum().T['group'].nlargest(num_top).index
  # print('total samples', temp.shape[0])

  #too many reciies without enough samples
  if temp.shape[0] < min_samples:
    top_score = 0
  else:
    top_score = max(filtered['note'])-2
    filtered_rating = filtered[filtered['note'] >= top_score]

  filtered_rating = filtered

  temp2 = expand_get_dummies_todf(col2, df = filtered_rating, split = ",").drop(top, axis=1)
  temp2['group'] = 'group'
  num = int((temp.shape[1])*include_magic)
  top_magic = temp2.groupby('group').sum().T['group'].nlargest(num)

  return top_magic

In [ ]:
df_main.columns

Index(['url', 'fiche_recette', 'titre', 'tags', 'tag_1', 'temps', 'cout',
       'difficulte', 'note', 'like', 'partage', 'utilisateur', 'type',
       'ingredients'],
      dtype='object')

In [ ]:
df_main['ingred_clean_lemma'] = df_main['ingredients'].apply(lemma)
df_main['ingred_clean'] = df_main['ingred_clean_lemma'].apply(remove_stop)

In [ ]:
get_top_percent(data=df_main, col='ingred_clean', percent=.10, tag='accompagnement')

sel                       56
poivre                    48
ail                       25
pomme terre               24
beurre                    22
farine                    21
oignon                    21
oeuf                      20
huile olive               19
huile                     14
eau                       13
muscade                   12
persil                    12
crème liquide              9
kilogramme pomme terre     9
lait                       9
sucre                      9
thym                       8
courgette                  7
Name: group, dtype: uint8

In [ ]:
find_magic(data=df_main, col='titre', col2='ingred_clean', search='poulet', exclude_top=.2, include_magic=.1, min_samples=10)

tomate              3
bacon               2
basilic             2
concentré tomate    2
coriandr            2
curcuma             2
curry               2
farine              2
jambon              2
maïzena             2
miel                2
muscade             2
paprika             2
poitrine fumée      2
pâte feuilletée     2
Name: group, dtype: uint8